In [3]:
# 한 줄로 설치
!pip install selenium pandas openpyxl webdriver-manager

# # 또는 하나씩
# pip install selenium
# pip install pandas
# pip install openpyxl
# pip install webdriver-manager

  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -------- ------------------------------- 2.1/9.7 MB 11.2 MB/s eta 0:00:01
   ---------------------- ----------------- 5.5/9.7 MB 14.2 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.7 MB 15.9 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 15.3 MB/s eta 0:00:00
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)

  Attempting uninstall: urllib3

    Found existing installation: urllib3 2.3.0

    Uninstalling urllib3-2.3.0:

      Successfully uninstalled urllib3-2.3.0

   ----- ---------------------------------- 1/8 [urllib3]
   ----- ---------------------------------- 1/8 [urllib3]
  Attempting uninstall: typing_extensions
   ----- ---------------------------------- 1/8 [urllib3]
    Found existing installation: typing_extensions 4.12.2
   ----- ---------------------------------- 1/8 [u

In [ ]:
"""
올리브영 글로벌 미국 - 샘플 크롤링
기본 제품 정보만 수집 (국가, 대분류, 중분류, 순위, 제품명, 브랜드)
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from datetime import datetime

class OliveYoungSampleCrawler:
    def __init__(self, headless=False):
        """
        샘플 크롤러 초기화
        """
        from selenium.webdriver.chrome.service import Service
        from webdriver_manager.chrome import ChromeDriverManager
        
        print("=" * 70)
        print("올리브영 글로벌 샘플 크롤러 시작")
        print("=" * 70)
        
        options = webdriver.ChromeOptions()
        if headless:
            options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
        
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=options)
        self.wait = WebDriverWait(self.driver, 10)
        
        # 카테고리 정의 (11개)
        self.categories = {
            'Skincare': {
                'All Skincare': '1000000008',
                'Moisturizers': '1000000009',
                'Cleansers': '1000000010'
            },
            'Makeup': {
                'All Makeup': '1000000031',
                'Face': '1000000032',
                'Eye': '1000000040',
                'Lip': '1000000045'
            },
            'Hair': {
                'All Hair': '1000000070'
            },
            'Mask': {
                'All Face Masks': '1000000003'
            },
            'Suncare': {
                'All Suncare': '1000000011'
            }
        }
        
        self.country = 'USA'  # 미국 고정
        
    def scroll_and_wait(self, scroll_count=2):
        """페이지 스크롤하여 제품 로드"""
        for i in range(scroll_count):
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
    
    def extract_products_from_category(self, major_category, sub_category, category_id, limit=30):
        """
        카테고리에서 제품 기본 정보 추출 (TOP 30)
        """
        url = f"https://global.oliveyoung.com/display/category?ctgrNo={category_id}"
        
        print(f"\n{'='*70}")
        print(f"대분류: {major_category} | 중분류: {sub_category}")
        print(f"URL: {url}")
        print(f"{'='*70}")
        
        try:
            self.driver.get(url)
            time.sleep(3)
            
            # 스크롤하여 더 많은 제품 로드
            self.scroll_and_wait(scroll_count=3)
            
            products = []
            
            # 제품 요소 찾기 - 여러 가능한 선택자 시도
            product_selectors = [
                'li[class*="product"]',
                'div[class*="product"]',
                'a[href*="/product/detail"]'
            ]
            
            product_elements = []
            for selector in product_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if elements:
                        product_elements = elements
                        print(f"✓ 선택자 '{selector}'로 {len(elements)}개 요소 발견")
                        break
                except:
                    continue
            
            if not product_elements:
                print(f"⚠ 제품을 찾을 수 없습니다.")
                return products
            
            print(f"발견된 제품 수: {len(product_elements)}")
            
            # TOP 30개 제품만 처리
            for rank, element in enumerate(product_elements[:limit], 1):
                try:
                    product_data = {
                        'country': self.country,
                        'major_category': major_category,
                        'sub_category': sub_category,
                        'rank': rank,
                        'product_name': '',
                        'brand': ''
                    }
                    
                    # 제품명 추출 시도 (여러 선택자)
                    name_selectors = [
                        '[class*="name"]',
                        '[class*="title"]',
                        'span[class*="prd"]',
                        'p[class*="prd"]'
                    ]
                    
                    for selector in name_selectors:
                        try:
                            name_elem = element.find_element(By.CSS_SELECTOR, selector)
                            name_text = name_elem.text.strip()
                            if name_text and len(name_text) > 3:  # 최소 3글자 이상
                                product_data['product_name'] = name_text
                                break
                        except:
                            continue
                    
                    # 브랜드 추출 시도
                    brand_selectors = [
                        '[class*="brand"]',
                        'span[class*="brand"]',
                        'p[class*="brand"]'
                    ]
                    
                    for selector in brand_selectors:
                        try:
                            brand_elem = element.find_element(By.CSS_SELECTOR, selector)
                            brand_text = brand_elem.text.strip()
                            if brand_text:
                                product_data['brand'] = brand_text
                                break
                        except:
                            continue
                    
                    # 제품명이 있는 경우만 추가
                    if product_data['product_name']:
                        products.append(product_data)
                        print(f"  [{rank:2d}] {product_data['brand']:20s} | {product_data['product_name'][:50]}")
                    
                except Exception as e:
                    print(f"  [오류] 제품 {rank} 추출 실패: {str(e)[:50]}")
                    continue
            
            print(f"\n✓ 수집 완료: {len(products)}개 제품")
            return products
            
        except Exception as e:
            print(f"✗ 카테고리 크롤링 오류: {e}")
            return []
    
    def crawl_all_categories(self):
        """
        모든 카테고리 크롤링
        """
        all_products = []
        total_categories = sum(len(subs) for subs in self.categories.values())
        current = 0
        
        print(f"\n총 {total_categories}개 카테고리 크롤링 시작...")
        print(f"예상 소요 시간: 약 {total_categories * 2}분\n")
        
        start_time = time.time()
        
        for major_category, sub_categories in self.categories.items():
            for sub_category, category_id in sub_categories.items():
                current += 1
                print(f"\n진행: [{current}/{total_categories}]")
                
                # 제품 수집
                products = self.extract_products_from_category(
                    major_category, 
                    sub_category, 
                    category_id, 
                    limit=30
                )
                
                all_products.extend(products)
                
                # Rate limiting
                time.sleep(2)
        
        elapsed_time = time.time() - start_time
        print(f"\n{'='*70}")
        print(f"크롤링 완료!")
        print(f"소요 시간: {elapsed_time/60:.1f}분")
        print(f"수집된 제품 수: {len(all_products)}개")
        print(f"{'='*70}\n")
        
        return all_products
    
    def save_to_excel(self, data, filename='oliveyoung_sample_usa.xlsx'):
        """
        데이터를 Excel로 저장
        """
        if not data:
            print("⚠ 저장할 데이터가 없습니다.")
            return
        
        df = pd.DataFrame(data)
        
        # 컬럼 순서 정렬
        columns_order = ['country', 'major_category', 'sub_category', 'rank', 'product_name', 'brand']
        df = df[columns_order]
        
        # Excel 저장
        df.to_excel(filename, index=False, engine='openpyxl')
        
        print(f"✓ Excel 파일 저장: {filename}")
        
        # 통계 출력
        print(f"\n{'='*70}")
        print("데이터 통계:")
        print(f"{'='*70}")
        print(f"총 제품 수: {len(df)}")
        print(f"\n대분류별 제품 수:")
        print(df['major_category'].value_counts())
        print(f"\n브랜드별 제품 수 (TOP 10):")
        print(df['brand'].value_counts().head(10))
        
        # 아모레퍼시픽 브랜드 확인
        amore_brands = ['LANEIGE', 'SULWHASOO', 'MAMONDE', 'ETUDE', 'INNISFREE', 
                       'HERA', 'IOPE', 'PRIMERA', 'AMOREPACIFIC']
        
        amore_products = df[df['brand'].str.upper().isin([b.upper() for b in amore_brands])]
        
        print(f"\n{'='*70}")
        print("아모레퍼시픽 브랜드 제품:")
        print(f"{'='*70}")
        if len(amore_products) > 0:
            print(f"✓ 발견: {len(amore_products)}개")
            print(amore_products[['brand', 'major_category', 'product_name']].to_string(index=False))
        else:
            print("⚠ 아모레퍼시픽 브랜드 제품이 발견되지 않았습니다.")
        
    def close(self):
        """브라우저 종료"""
        self.driver.quit()

def main():
    """
    메인 실행 함수
    """
    print("\n" + "="*70)
    print("올리브영 글로벌 미국 샘플 크롤러")
    print("수집 항목: 국가, 대분류, 중분류, 순위, 제품명, 브랜드")
    print("="*70 + "\n")
    
    # 크롤러 초기화 (headless=False로 브라우저 표시)
    crawler = OliveYoungSampleCrawler(headless=False)
    
    try:
        # 크롤링 실행
        products = crawler.crawl_all_categories()
        
        # 데이터 저장
        if products:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f'oliveyoung_sample_usa_{timestamp}.xlsx'
            crawler.save_to_excel(products, filename)
        else:
            print("⚠ 수집된 데이터가 없습니다.")
        
    except KeyboardInterrupt:
        print("\n\n사용자에 의해 중단되었습니다.")
    except Exception as e:
        print(f"\n오류 발생: {e}")
    finally:
        # 브라우저 종료
        crawler.close()
        print("\n크롤러 종료")

if __name__ == "__main__":
    main()


올리브영 글로벌 미국 샘플 크롤러
수집 항목: 국가, 대분류, 중분류, 순위, 제품명, 브랜드

올리브영 글로벌 샘플 크롤러 시작

총 10개 카테고리 크롤링 시작...
예상 소요 시간: 약 20분


진행: [1/10]

대분류: Skincare | 중분류: All Skincare
URL: https://global.oliveyoung.com/display/category?ctgrNo=1000000008
✓ 선택자 'div[class*="product"]'로 2개 요소 발견
발견된 제품 수: 2

✓ 수집 완료: 0개 제품

진행: [2/10]

대분류: Skincare | 중분류: Moisturizers
URL: https://global.oliveyoung.com/display/category?ctgrNo=1000000009
✓ 선택자 'div[class*="product"]'로 2개 요소 발견
발견된 제품 수: 2

✓ 수집 완료: 0개 제품

진행: [3/10]

대분류: Skincare | 중분류: Cleansers
URL: https://global.oliveyoung.com/display/category?ctgrNo=1000000010
✓ 선택자 'div[class*="product"]'로 2개 요소 발견
발견된 제품 수: 2

✓ 수집 완료: 0개 제품

진행: [4/10]

대분류: Makeup | 중분류: All Makeup
URL: https://global.oliveyoung.com/display/category?ctgrNo=1000000031
✓ 선택자 'div[class*="product"]'로 2개 요소 발견
발견된 제품 수: 2

✓ 수집 완료: 0개 제품

진행: [5/10]

대분류: Makeup | 중분류: Face
URL: https://global.oliveyoung.com/display/category?ctgrNo=1000000032
✓ 선택자 'div[class*="product"]'로 2개 요소 발견
발견된 제품 

In [8]:
"""
올리브영 HTML 구조 분석 도구
실제 페이지의 HTML을 파일로 저장해서 구조 확인
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def analyze_oliveyoung_structure():
    """
    올리브영 페이지 HTML 구조 분석
    """
    print("=" * 70)
    print("올리브영 HTML 구조 분석 시작")
    print("=" * 70)
    
    # Chrome 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        # 올리브영 Skincare 카테고리 열기
        url = "https://global.oliveyoung.com/display/category?ctgrNo=1000000008"
        print(f"\n페이지 로딩 중: {url}")
        driver.get(url)
        
        # 페이지 로드 대기
        print("페이지 로드 대기 중... (10초)")
        time.sleep(10)
        
        # 스크롤
        print("스크롤 중...")
        for i in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
        
        # HTML 저장
        html_content = driver.page_source
        
        # 현재 디렉토리에 저장
        filename = "oliveyoung_page_structure.html"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        print(f"\n✓ HTML 저장 완료: {filename}")
        print("\n다음 단계:")
        print("1. 저장된 HTML 파일을 브라우저로 열기")
        print("2. F12 눌러서 개발자 도구 열기")
        print("3. 제품 하나 선택해서 HTML 구조 확인")
        print("4. class 이름들 확인해서 알려주기")
        
        # 스크린샷도 저장
        screenshot_file = "oliveyoung_screenshot.png"
        driver.save_screenshot(screenshot_file)
        print(f"✓ 스크린샷 저장: {screenshot_file}")
        
        # 간단한 구조 분석
        print("\n" + "=" * 70)
        print("페이지 구조 빠른 분석:")
        print("=" * 70)
        
        from selenium.webdriver.common.by import By
        
        # 가능한 선택자들 테스트
        test_selectors = [
            ('제품 리스트 컨테이너', 'ul[class*="list"]'),
            ('제품 아이템 (li)', 'li[class*="item"]'),
            ('제품 아이템 (div)', 'div[class*="item"]'),
            ('제품 링크', 'a[href*="/product/detail"]'),
            ('이름 필드', '[class*="name"]'),
            ('브랜드 필드', '[class*="brand"]'),
            ('가격 필드', '[class*="price"]'),
        ]
        
        for desc, selector in test_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                print(f"{desc:30s} | {selector:40s} | 발견: {len(elements)}개")
                
                # 첫 번째 요소의 텍스트 출력
                if elements and len(elements) > 0:
                    text = elements[0].text.strip()[:50]
                    if text:
                        print(f"  └─ 첫 번째 요소 텍스트: {text}")
            except:
                print(f"{desc:30s} | {selector:40s} | 오류")
        
    finally:
        driver.quit()
        print("\n분석 완료!")

if __name__ == "__main__":
    analyze_oliveyoung_structure()

올리브영 HTML 구조 분석 시작

페이지 로딩 중: https://global.oliveyoung.com/display/category?ctgrNo=1000000008
페이지 로드 대기 중... (10초)
스크롤 중...

✓ HTML 저장 완료: oliveyoung_page_structure.html

다음 단계:
1. 저장된 HTML 파일을 브라우저로 열기
2. F12 눌러서 개발자 도구 열기
3. 제품 하나 선택해서 HTML 구조 확인
4. class 이름들 확인해서 알려주기
✓ 스크린샷 저장: oliveyoung_screenshot.png

페이지 구조 빠른 분석:
제품 리스트 컨테이너                    | ul[class*="list"]                        | 발견: 69개
제품 아이템 (li)                    | li[class*="item"]                        | 발견: 861개
제품 아이템 (div)                   | div[class*="item"]                       | 발견: 6개
제품 링크                          | a[href*="/product/detail"]               | 발견: 48개
이름 필드                          | [class*="name"]                          | 발견: 222개
브랜드 필드                         | [class*="brand"]                         | 발견: 328개
가격 필드                          | [class*="price"]                         | 발견: 32개

분석 완료!


# main 크롤링

In [1]:
"""
올리브영 글로벌 크롤러 (최종 완성판)
- 15개 컬럼 (할인율, 세부평점 포함)
- 모든 선택자 수정 완료
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from datetime import datetime
import os
import json

class OliveYoungCrawler:
    def __init__(self, save_folder='./oliveyoung_data'):
        """크롤러 초기화"""
        from selenium.webdriver.chrome.service import Service
        from webdriver_manager.chrome import ChromeDriverManager
        
        print("=" * 70)
        print("올리브영 글로벌 크롤러 (최종 완성판)")
        print("=" * 70)
        
        # 저장 폴더
        self.save_folder = save_folder
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        print(f"✓ 저장 폴더: {os.path.abspath(save_folder)}")
        
        # Chrome 설정
        options = webdriver.ChromeOptions()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
        
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=options)
        self.wait = WebDriverWait(self.driver, 10)
        
        # 데이터 저장
        self.all_data = []
        
        # 파일명
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.excel_filename = os.path.join(save_folder, f'oliveyoung_data_{timestamp}.xlsx')
        
        self.country = 'USA'
    
    def get_product_urls(self, category_url, limit=30):
        """제품 URL 수집 (View 36)"""
        print(f"\n{'='*70}")
        print(f"카테고리 페이지 접속...")
        print(f"{'='*70}")
        
        try:
            # View 36 설정
            if '?' in category_url:
                url_with_view = f"{category_url}&pageSize=36"
            else:
                url_with_view = f"{category_url}?pageSize=36"
            
            self.driver.get(url_with_view)
            time.sleep(5)
            
            # 36개 표시 버튼 클릭 시도
            try:
                view_buttons = self.driver.find_elements(By.CSS_SELECTOR, 'button, a')
                for btn in view_buttons:
                    if '36' in btn.text:
                        btn.click()
                        time.sleep(2)
                        print("✓ View 36 클릭")
                        break
            except:
                pass
            
            # 스크롤
            for i in range(3):
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.5)
            
            # 제품 링크 수집
            product_links = self.driver.find_elements(By.CSS_SELECTOR, 'a[href*="/product/detail"]')
            
            urls = []
            for link in product_links:
                href = link.get_attribute('href')
                if href and href not in urls:
                    urls.append(href)
            
            print(f"✓ 수집된 제품 URL: {len(urls)}개")
            return urls[:limit]
            
        except Exception as e:
            print(f"✗ URL 수집 오류: {e}")
            return []
    
    def extract_product_and_reviews(self, product_url, rank, major_category, sub_category):
        """제품 정보 + 리뷰 통합 수집"""
        all_rows = []
        
        try:
            self.driver.get(product_url)
            time.sleep(3)
            
            # === 제품 기본 정보 ===
            product_info = {
                'country': self.country,
                'major_category': major_category,
                'sub_category': sub_category,
                'rank': rank,
                'product_url': product_url
            }
            
            # 제품명
            try:
                product_info['product_name'] = self.driver.find_element(
                    By.CSS_SELECTOR, 'dt[data-testid="product-name"]'
                ).text.strip()
            except:
                product_info['product_name'] = ''
            
            # 브랜드 (dt.notranslate만)
            try:
                brand_elem = self.driver.find_element(By.CSS_SELECTOR, 'dt.notranslate')
                product_info['brand'] = brand_elem.text.strip()
            except:
                product_info['brand'] = ''
            
            # 할인율
            try:
                discount = self.driver.find_element(By.CSS_SELECTOR, 'span.discount-rate').text.strip()
                product_info['discount_rate'] = discount
            except:
                product_info['discount_rate'] = ''
            
            # 가격 (원가)
            try:
                price_elem = self.driver.find_element(By.CSS_SELECTOR, 'dt.price')
                # dt.price > div > span에서 원가 추출
                price_spans = price_elem.find_elements(By.CSS_SELECTOR, 'div > span')
                if price_spans:
                    product_info['price'] = price_spans[0].text.strip()
                else:
                    product_info['price'] = ''
            except:
                product_info['price'] = ''
            
            # 평균평점 (4.7 같은 숫자)
            try:
                rating_elem = self.driver.find_element(By.CSS_SELECTOR, 'dl.prd-rating-info')
                # span 중에서 숫자가 있는 것 찾기
                rating_spans = rating_elem.find_elements(By.TAG_NAME, 'span')
                for span in rating_spans:
                    text = span.text.strip()
                    if text and text[0].isdigit():  # 숫자로 시작하면
                        product_info['rating'] = text
                        break
                if 'rating' not in product_info:
                    product_info['rating'] = ''
            except:
                product_info['rating'] = ''
            
            # 성분 (버튼 클릭 후 추출)
            try:
                ingr_button = self.driver.find_element(
                    By.CSS_SELECTOR, 'a[data-testid="product-featuredingredients-link"]'
                )
                ingr_button.click()
                time.sleep(2)
                
                product_info['ingredients'] = self.driver.find_element(
                    By.CSS_SELECTOR, 'div[data-testid="product-featuredingredients-content"]'
                ).text.strip()
            except:
                product_info['ingredients'] = ''
            
            # 제품설명 (버튼 클릭 후 추출)
            try:
                desc_button = self.driver.find_element(
                    By.CSS_SELECTOR, 'a[data-testid="product-whyweloveit-link"]'
                )
                desc_button.click()
                time.sleep(2)
                
                product_info['description'] = self.driver.find_element(
                    By.CSS_SELECTOR, 'div[data-testid="product-whyweloveit-content"]'
                ).text.strip()
            except:
                product_info['description'] = ''

            print(f"  ✓ [{rank:2d}] {product_info['brand']:20s} | {product_info['product_name'][:50]}")
            print(f"       가격: {product_info['price']} (할인: {product_info['discount_rate']}) | 평점: {product_info['rating']}")
            
            # === 리뷰 수집 ===
            print(f"    📜 리뷰 수집 시작...")
            
            # 리뷰 섹션으로 스크롤
            time.sleep(2)
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
            time.sleep(2)
            
            # Newest 버튼 클릭
            try:
                newest_buttons = self.driver.find_elements(By.CSS_SELECTOR, 'li[tabindex="0"]')
                for button in newest_buttons:
                    if 'Newest' in button.text or 'newest' in button.text:
                        button.click()
                        time.sleep(2)
                        print(f"    ✓ 최신순 정렬")
                        break
            except:
                print(f"    ⚠ 최신순 버튼 찾기 실패")
            
            # More 버튼 반복 클릭
            more_clicks = 0
            while more_clicks < 10:
                try:
                    more_btn = self.driver.find_element(By.CSS_SELECTOR, 'button.review-list-more-btn')
                    if more_btn.is_displayed():
                        self.driver.execute_script("arguments[0].scrollIntoView(true);", more_btn)
                        time.sleep(1)
                        more_btn.click()
                        more_clicks += 1
                        print(f"    📝 More 버튼 클릭 ({more_clicks})")
                        time.sleep(2)
                        
                        # 리뷰 개수 확인
                        reviews = self.driver.find_elements(By.CSS_SELECTOR, 'div.product-review-unit-main')
                        if len(reviews) >= 100:
                            print(f"    ✓ 목표 달성: {len(reviews)}개")
                            break
                    else:
                        break
                except NoSuchElementException:
                    print(f"    ✓ More 버튼 없음")
                    break
                except:
                    break
            
            # 리뷰 파싱
            review_elements = self.driver.find_elements(By.CSS_SELECTOR, 'div.product-review-unit-main')
            print(f"    ✓ 발견된 리뷰: {len(review_elements)}개")
            
            for idx, review_elem in enumerate(review_elements[:100], 1):
                try:
                    row = product_info.copy()
                    
                    # 리뷰 전체 별점 (1-5)
                    try:
                        # filled 클래스가 있는 별 개수 세기
                        filled_stars = review_elem.find_elements(
                            By.CSS_SELECTOR, 'div.product-review-unit-header div.review-star-rating div.icon-star.filled'
                        )
                        # 10개 중 filled 개수를 5점 만점으로 환산
                        row['review_rating'] = len(filled_stars) / 2 if filled_stars else ''
                    except:
                        row['review_rating'] = ''
                    
                    # 리뷰 내용
                    try:
                        row['review_content'] = review_elem.find_element(
                            By.CSS_SELECTOR, 'div.review-unit-cont-comment'
                        ).text.strip()
                    except:
                        row['review_content'] = ''
                    
                    # 작성 날짜
                    try:
                        parent = review_elem.find_element(By.XPATH, '..')
                        date_elem = parent.find_element(
                            By.CSS_SELECTOR, 'span.review-write-info-date.notranslate'
                        )
                        row['review_date'] = date_elem.get_attribute('textContent').strip()
                    except:
                        row['review_date'] = ''
                    
                    # 세부 평점 (카테고리별)
                    try:
                        detail_ratings = {}
                        # ul.list-review-evlt > li 찾기
                        detail_items = review_elem.find_elements(By.CSS_SELECTOR, 'ul.list-review-evlt > li')
                        
                        for item in detail_items:
                            try:
                                # 항목명
                                category_name = item.find_element(By.TAG_NAME, 'span').text.strip()
                                
                                # filled 별 개수 세기
                                filled = item.find_elements(By.CSS_SELECTOR, 'div.icon-star.filled')
                                # 10개 중 filled 개수를 5점 만점으로 환산
                                rating_value = len(filled) / 2
                                
                                if category_name:
                                    detail_ratings[category_name] = rating_value
                            except:
                                continue
                        
                        # JSON 문자열로 저장
                        row['review_detail_ratings'] = json.dumps(detail_ratings) if detail_ratings else ''
                    except:
                        row['review_detail_ratings'] = ''
                    
                    all_rows.append(row)
                    
                    if idx % 20 == 0:
                        print(f"    📝 리뷰 수집 중... {idx}/{min(100, len(review_elements))}")
                
                except:
                    continue
            
            print(f"    ✓ 수집 완료: {len(all_rows)}개 리뷰")
            return all_rows
            
        except Exception as e:
            print(f"  ✗ 오류: {str(e)[:50]}")
            return all_rows
    
    def save_to_excel(self):
        """Excel 저장"""
        if not self.all_data:
            print("⚠ 저장할 데이터 없음")
            return
        
        df = pd.DataFrame(self.all_data)
        
        # 컬럼 순서 (15개)
        columns = [
            'country', 'major_category', 'sub_category', 'rank',
            'product_name', 'brand', 'discount_rate', 'price', 'rating',
            'ingredients', 'description',
            'review_content', 'review_date', 'review_rating', 'review_detail_ratings',
            'product_url'
        ]
        
        existing_cols = [c for c in columns if c in df.columns]
        df = df[existing_cols]
        
        df.to_excel(self.excel_filename, index=False, engine='openpyxl')
        print(f"\n💾 저장: {self.excel_filename} ({len(df)}행)")
    
    def crawl_category(self, category_url, major_category, sub_category, limit=30):
        """카테고리 크롤링"""
        print(f"\n{'='*70}")
        print(f"대분류: {major_category} | 중분류: {sub_category}")
        print(f"{'='*70}")
        
        start_time = time.time()
        
        # 1. 제품 URL 수집
        urls = self.get_product_urls(category_url, limit)
        
        if not urls:
            print("⚠ URL 없음")
            return
        
        # 2. 각 제품 크롤링
        for rank, url in enumerate(urls, 1):
            print(f"\n📦 제품 [{rank}/{len(urls)}]")
            
            rows = self.extract_product_and_reviews(
                url, rank, major_category, sub_category
            )
            
            self.all_data.extend(rows)
            
            # 중간 저장 (5개마다)
            if rank % 5 == 0:
                self.save_to_excel()
                print(f"  💾 중간 저장 완료 ({len(self.all_data)}행)")
            
            time.sleep(2)
        
        # 최종 저장
        self.save_to_excel()
        
        elapsed = time.time() - start_time
        print(f"\n{'='*70}")
        print(f"✓ 완료! ({elapsed/60:.1f}분)")
        print(f"총 {len(self.all_data)}행")
        print(f"{'='*70}")
    
    def print_stats(self):
        """통계"""
        if not self.all_data:
            return
        
        df = pd.DataFrame(self.all_data)
        print(f"\n{'='*70}")
        print("📊 통계")
        print(f"{'='*70}")
        print(f"총 행: {len(df)}")
        print(f"제품 수: {df['product_name'].nunique()}")
        print(f"\n브랜드 TOP 10:")
        print(df['brand'].value_counts().head(10))
        
        # 평균 평점
        if 'rating' in df.columns:
            try:
                ratings = df['rating'].replace('', None).dropna()
                if len(ratings) > 0:
                    avg_rating = pd.to_numeric(ratings, errors='coerce').mean()
                    print(f"\n평균 제품 평점: {avg_rating:.2f}")
            except:
                pass
    
    def close(self):
        """종료"""
        self.driver.quit()

def main():
    """실행"""
    print("\n" + "="*70)
    print("올리브영 글로벌 크롤러 (최종판)")
    print("Skincare - All Skincare (TOP 30)")
    print("="*70 + "\n")
    
    crawler = OliveYoungCrawler(save_folder='./oliveyoung_data')
    
    try:
        crawler.crawl_category(
            category_url="https://global.oliveyoung.com/display/category?ctgrNo=1000000008",
            major_category='Skincare',
            sub_category='All Skincare',
            limit=30
        )
        
        crawler.print_stats()
        
    except KeyboardInterrupt:
        print("\n⚠ 중단")
        crawler.save_to_excel()
        crawler.print_stats()
    except Exception as e:
        print(f"\n✗ 오류: {e}")
        import traceback
        traceback.print_exc()
    finally:
        crawler.close()
        print("\n종료")

if __name__ == "__main__":
    main()


올리브영 글로벌 크롤러 (최종판)
Skincare - All Skincare (TOP 30)

올리브영 글로벌 크롤러 (최종 완성판)
✓ 저장 폴더: c:\Users\cub72\Desktop\공모전\아모퍼\oliveyoung_data

대분류: Skincare | 중분류: All Skincare

카테고리 페이지 접속...
✓ View 36 클릭
✓ 수집된 제품 URL: 36개

📦 제품 [1/30]
  ✓ [ 1] Anua                 | ★2025 Awards★ Anua Niacinamide 10 TXA 4 Dark Spot 
       가격: US$74.00 (할인: 59%) | 평점: 4.8
    📜 리뷰 수집 시작...

⚠ 중단
⚠ 저장할 데이터 없음

종료


In [2]:
"""
Olive Young Global - Web Crawler
K-Beauty 제품 및 리뷰 데이터 수집 크롤러

수집 데이터: 제품 정보, 가격, 평점, 성분, 리뷰(최신 100개)
대상 시장: USA, Japan, China
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from datetime import datetime
import os
import json


class OliveYoungCrawler:
    """
    올리브영 글로벌 웹사이트 크롤러
    
    Features:
    - 제품 상세 정보 수집 (제품명, 브랜드, 가격, 평점, 성분, 설명)
    - 리뷰 데이터 수집 (내용, 날짜, 평점, 카테고리별 세부평점)
    - 실시간 저장 (5개 제품마다)
    - View 36 자동 설정
    """
    
    def __init__(self, save_folder='./oliveyoung_data'):
        """크롤러 초기화"""
        from selenium.webdriver.chrome.service import Service
        from webdriver_manager.chrome import ChromeDriverManager
        
        print("=" * 70)
        print("Olive Young Global Crawler - Initialization")
        print("=" * 70)
        
        # 저장 폴더 설정
        self.save_folder = save_folder
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        print(f"[INFO] 저장 폴더: {os.path.abspath(save_folder)}")
        
        # Chrome 드라이버 설정
        options = webdriver.ChromeOptions()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
        
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=options)
        self.wait = WebDriverWait(self.driver, 10)
        
        # 데이터 저장 리스트
        self.all_data = []
        
        # 파일명 생성
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.excel_filename = os.path.join(save_folder, f'oliveyoung_data_{timestamp}.xlsx')
        
        self.country = 'USA'
    
    def get_product_urls(self, category_url, limit=30):
        """
        카테고리 페이지에서 제품 URL 수집
        
        Args:
            category_url (str): 카테고리 페이지 URL
            limit (int): 수집할 제품 개수 (기본: 30)
        
        Returns:
            list: 제품 상세 페이지 URL 리스트
        """
        print(f"\n{'='*70}")
        print(f"[STEP 1] 카테고리 페이지 접속 중...")
        print(f"{'='*70}")
        
        try:
            # View 36 파라미터 추가
            if '?' in category_url:
                url_with_view = f"{category_url}&pageSize=36"
            else:
                url_with_view = f"{category_url}?pageSize=36"
            
            self.driver.get(url_with_view)
            time.sleep(5)
            
            # View 36 버튼 클릭 시도
            try:
                view_buttons = self.driver.find_elements(By.CSS_SELECTOR, 'button, a')
                for btn in view_buttons:
                    if '36' in btn.text:
                        btn.click()
                        time.sleep(2)
                        print("[SUCCESS] View 36 설정 완료")
                        break
            except:
                pass
            
            # 페이지 스크롤
            for i in range(3):
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.5)
            
            # 제품 링크 수집
            product_links = self.driver.find_elements(By.CSS_SELECTOR, 'a[href*="/product/detail"]')
            
            urls = []
            for link in product_links:
                href = link.get_attribute('href')
                if href and href not in urls:
                    urls.append(href)
            
            print(f"[SUCCESS] 제품 URL 수집 완료: {len(urls)}개")
            return urls[:limit]
            
        except Exception as e:
            print(f"[ERROR] URL 수집 실패: {e}")
            return []
    
    def extract_product_and_reviews(self, product_url, rank, major_category, sub_category):
        """
        제품 상세 정보 및 리뷰 수집
        
        Args:
            product_url (str): 제품 상세 페이지 URL
            rank (int): 카테고리 내 순위
            major_category (str): 대분류 카테고리
            sub_category (str): 중분류 카테고리
        
        Returns:
            list: 제품 정보 + 리뷰가 결합된 데이터 리스트
        """
        all_rows = []
        
        try:
            self.driver.get(product_url)
            time.sleep(3)
            
            # === 제품 기본 정보 수집 ===
            product_info = {
                'country': self.country,
                'major_category': major_category,
                'sub_category': sub_category,
                'rank': rank,
                'product_url': product_url
            }
            
            # 제품명
            try:
                product_info['product_name'] = self.driver.find_element(
                    By.CSS_SELECTOR, 'dt[data-testid="product-name"]'
                ).text.strip()
            except:
                product_info['product_name'] = ''
            
            # 브랜드
            try:
                brand_elem = self.driver.find_element(By.CSS_SELECTOR, 'dt.notranslate')
                product_info['brand'] = brand_elem.text.strip()
            except:
                product_info['brand'] = ''
            
            # 할인율
            try:
                discount = self.driver.find_element(By.CSS_SELECTOR, 'span.discount-rate').text.strip()
                product_info['discount_rate'] = discount
            except:
                product_info['discount_rate'] = ''
            
            # 가격 (원가)
            try:
                price_elem = self.driver.find_element(By.CSS_SELECTOR, 'dt.price')
                price_spans = price_elem.find_elements(By.CSS_SELECTOR, 'div > span')
                if price_spans:
                    product_info['price'] = price_spans[0].text.strip()
                else:
                    product_info['price'] = ''
            except:
                product_info['price'] = ''
            
            # 평균 평점
            try:
                rating_elem = self.driver.find_element(By.CSS_SELECTOR, 'dl.prd-rating-info')
                rating_spans = rating_elem.find_elements(By.TAG_NAME, 'span')
                for span in rating_spans:
                    text = span.text.strip()
                    if text and text[0].isdigit():
                        product_info['rating'] = text
                        break
                if 'rating' not in product_info:
                    product_info['rating'] = ''
            except:
                product_info['rating'] = ''
            
            # 성분 (버튼 클릭 후 수집)
            try:
                ingr_button = self.driver.find_element(
                    By.CSS_SELECTOR, 'a[data-testid="product-featuredingredients-link"]'
                )
                ingr_button.click()
                time.sleep(2)
                
                product_info['ingredients'] = self.driver.find_element(
                    By.CSS_SELECTOR, 'div[data-testid="product-featuredingredients-content"]'
                ).text.strip()
            except:
                product_info['ingredients'] = ''
            
            # 제품 설명 (버튼 클릭 후 수집)
            try:
                desc_button = self.driver.find_element(
                    By.CSS_SELECTOR, 'a[data-testid="product-whyweloveit-link"]'
                )
                desc_button.click()
                time.sleep(2)
                
                product_info['description'] = self.driver.find_element(
                    By.CSS_SELECTOR, 'div[data-testid="product-whyweloveit-content"]'
                ).text.strip()
            except:
                product_info['description'] = ''

            print(f"  [PRODUCT {rank:2d}] {product_info['brand']:20s} | {product_info['product_name'][:50]}")
            print(f"               가격: {product_info['price']} (할인: {product_info['discount_rate']}) | 평점: {product_info['rating']}")
            
            # === 리뷰 데이터 수집 ===
            print(f"    [STEP 2] 리뷰 수집 시작...")
            
            # 리뷰 섹션으로 스크롤
            time.sleep(2)
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
            time.sleep(2)
            
            # 최신순 정렬
            try:
                newest_buttons = self.driver.find_elements(By.CSS_SELECTOR, 'li[tabindex="0"]')
                for button in newest_buttons:
                    if 'Newest' in button.text or 'newest' in button.text:
                        button.click()
                        time.sleep(2)
                        print(f"    [SUCCESS] 최신순 정렬 완료")
                        break
            except:
                print(f"    [WARNING] 최신순 버튼 찾기 실패 (기본 정렬 사용)")
            
            # More 버튼 반복 클릭 (리뷰 100개 로드)
            more_clicks = 0
            while more_clicks < 10:
                try:
                    more_btn = self.driver.find_element(By.CSS_SELECTOR, 'button.review-list-more-btn')
                    if more_btn.is_displayed():
                        self.driver.execute_script("arguments[0].scrollIntoView(true);", more_btn)
                        time.sleep(1)
                        more_btn.click()
                        more_clicks += 1
                        print(f"    [LOADING] More 버튼 클릭 ({more_clicks}/10)")
                        time.sleep(2)
                        
                        # 리뷰 개수 확인
                        reviews = self.driver.find_elements(By.CSS_SELECTOR, 'div.product-review-unit-main')
                        if len(reviews) >= 100:
                            print(f"    [SUCCESS] 목표 달성: {len(reviews)}개 리뷰 로드 완료")
                            break
                    else:
                        break
                except NoSuchElementException:
                    print(f"    [INFO] More 버튼 없음 (모든 리뷰 로드 완료)")
                    break
                except:
                    break
            
            # 리뷰 파싱
            review_elements = self.driver.find_elements(By.CSS_SELECTOR, 'div.product-review-unit-main')
            print(f"    [INFO] 발견된 리뷰: {len(review_elements)}개")
            
            for idx, review_elem in enumerate(review_elements[:100], 1):
                try:
                    row = product_info.copy()
                    
                    # 리뷰 전체 별점 (1-5점)
                    try:
                        filled_stars = review_elem.find_elements(
                            By.CSS_SELECTOR, 'div.product-review-unit-header div.review-star-rating div.icon-star.filled'
                        )
                        row['review_rating'] = len(filled_stars) / 2 if filled_stars else ''
                    except:
                        row['review_rating'] = ''
                    
                    # 리뷰 내용
                    try:
                        row['review_content'] = review_elem.find_element(
                            By.CSS_SELECTOR, 'div.review-unit-cont-comment'
                        ).text.strip()
                    except:
                        row['review_content'] = ''
                    
                    # 작성 날짜
                    try:
                        parent = review_elem.find_element(By.XPATH, '..')
                        date_elem = parent.find_element(
                            By.CSS_SELECTOR, 'span.review-write-info-date.notranslate'
                        )
                        row['review_date'] = date_elem.get_attribute('textContent').strip()
                    except:
                        row['review_date'] = ''
                    
                    # 카테고리별 세부 평점
                    try:
                        detail_ratings = {}
                        detail_items = review_elem.find_elements(By.CSS_SELECTOR, 'ul.list-review-evlt > li')
                        
                        for item in detail_items:
                            try:
                                category_name = item.find_element(By.TAG_NAME, 'span').text.strip()
                                filled = item.find_elements(By.CSS_SELECTOR, 'div.icon-star.filled')
                                rating_value = len(filled) / 2
                                
                                if category_name:
                                    detail_ratings[category_name] = rating_value
                            except:
                                continue
                        
                        row['review_detail_ratings'] = json.dumps(detail_ratings) if detail_ratings else ''
                    except:
                        row['review_detail_ratings'] = ''
                    
                    all_rows.append(row)
                    
                    if idx % 20 == 0:
                        print(f"    [PROGRESS] 리뷰 수집 중... {idx}/{min(100, len(review_elements))}")
                
                except:
                    continue
            
            print(f"    [SUCCESS] 리뷰 수집 완료: {len(all_rows)}개")
            return all_rows
            
        except Exception as e:
            print(f"  [ERROR] 제품 처리 실패: {str(e)[:50]}")
            return all_rows
    
    def save_to_excel(self):
        """수집된 데이터를 Excel 파일로 저장"""
        if not self.all_data:
            print("[WARNING] 저장할 데이터가 없습니다.")
            return
        
        df = pd.DataFrame(self.all_data)
        
        # 컬럼 순서 정렬 (15개)
        columns = [
            'country', 'major_category', 'sub_category', 'rank',
            'product_name', 'brand', 'discount_rate', 'price', 'rating',
            'ingredients', 'description',
            'review_content', 'review_date', 'review_rating', 'review_detail_ratings',
            'product_url'
        ]
        
        existing_cols = [c for c in columns if c in df.columns]
        df = df[existing_cols]
        
        df.to_excel(self.excel_filename, index=False, engine='openpyxl')
        print(f"\n[SAVED] {self.excel_filename} ({len(df)}행)")
    
    def crawl_category(self, category_url, major_category, sub_category, limit=30):
        """
        카테고리 전체 크롤링 실행
        
        Args:
            category_url (str): 카테고리 페이지 URL
            major_category (str): 대분류 카테고리명
            sub_category (str): 중분류 카테고리명
            limit (int): 수집할 제품 개수
        """
        print(f"\n{'='*70}")
        print(f"[CATEGORY] {major_category} > {sub_category}")
        print(f"{'='*70}")
        
        start_time = time.time()
        
        # 1. 제품 URL 수집
        urls = self.get_product_urls(category_url, limit)
        
        if not urls:
            print("[ERROR] 제품 URL을 찾을 수 없습니다.")
            return
        
        # 2. 각 제품별 크롤링
        for rank, url in enumerate(urls, 1):
            print(f"\n[PROCESSING] 제품 [{rank}/{len(urls)}]")
            
            rows = self.extract_product_and_reviews(
                url, rank, major_category, sub_category
            )
            
            self.all_data.extend(rows)
            
            # 중간 저장 (5개마다)
            if rank % 5 == 0:
                self.save_to_excel()
                print(f"  [CHECKPOINT] 중간 저장 완료 (총 {len(self.all_data)}행)")
            
            time.sleep(2)
        
        # 최종 저장
        self.save_to_excel()
        
        elapsed = time.time() - start_time
        print(f"\n{'='*70}")
        print(f"[COMPLETED] 크롤링 완료 (소요시간: {elapsed/60:.1f}분)")
        print(f"[RESULT] 총 {len(self.all_data)}행 수집")
        print(f"{'='*70}")
    
    def print_stats(self):
        """수집된 데이터 통계 출력"""
        if not self.all_data:
            return
        
        df = pd.DataFrame(self.all_data)
        print(f"\n{'='*70}")
        print("[STATISTICS] 데이터 통계")
        print(f"{'='*70}")
        print(f"총 데이터 행: {len(df)}")
        print(f"제품 수: {df['product_name'].nunique()}")
        print(f"\n브랜드별 제품 수 (TOP 10):")
        print(df['brand'].value_counts().head(10))
        
        # 평균 평점 계산
        if 'rating' in df.columns:
            try:
                ratings = df['rating'].replace('', None).dropna()
                if len(ratings) > 0:
                    avg_rating = pd.to_numeric(ratings, errors='coerce').mean()
                    print(f"\n평균 제품 평점: {avg_rating:.2f}")
            except:
                pass
    
    def close(self):
        """브라우저 종료"""
        self.driver.quit()


def main():
    """메인 실행 함수"""
    print("\n" + "="*70)
    print("Olive Young Global Web Crawler")
    print("Target: Skincare - All Skincare (TOP 30)")
    print("="*70 + "\n")
    
    crawler = OliveYoungCrawler(save_folder='./oliveyoung_data')
    
    try:
        crawler.crawl_category(
            category_url="https://global.oliveyoung.com/display/category?ctgrNo=1000000008",
            major_category='Skincare',
            sub_category='All Skincare',
            limit=30
        )
        
        crawler.print_stats()
        
    except KeyboardInterrupt:
        print("\n[INTERRUPTED] 사용자에 의해 중단되었습니다.")
        crawler.save_to_excel()
        crawler.print_stats()
    except Exception as e:
        print(f"\n[ERROR] 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    finally:
        crawler.close()
        print("\n[EXIT] 프로그램 종료")


if __name__ == "__main__":
    main()


Olive Young Global Web Crawler
Target: Skincare - All Skincare (TOP 30)

Olive Young Global Crawler - Initialization
[INFO] 저장 폴더: c:\Users\cub72\Desktop\공모전\아모퍼\oliveyoung_data

[CATEGORY] Skincare > All Skincare

[STEP 1] 카테고리 페이지 접속 중...
[SUCCESS] View 36 설정 완료
[SUCCESS] 제품 URL 수집 완료: 36개

[PROCESSING] 제품 [1/30]
  [PRODUCT  1] Anua                 | ★2025 Awards★ Anua Niacinamide 10 TXA 4 Dark Spot 
               가격: US$74.00 (할인: 59%) | 평점: 4.8
    [STEP 2] 리뷰 수집 시작...
    [LOADING] More 버튼 클릭 (1/10)
    [LOADING] More 버튼 클릭 (2/10)
    [LOADING] More 버튼 클릭 (3/10)
    [LOADING] More 버튼 클릭 (4/10)
    [LOADING] More 버튼 클릭 (5/10)
    [LOADING] More 버튼 클릭 (6/10)
    [LOADING] More 버튼 클릭 (7/10)
    [LOADING] More 버튼 클릭 (8/10)
    [LOADING] More 버튼 클릭 (9/10)
    [SUCCESS] 목표 달성: 100개 리뷰 로드 완료
    [INFO] 발견된 리뷰: 100개
    [PROGRESS] 리뷰 수집 중... 20/100
    [PROGRESS] 리뷰 수집 중... 40/100
    [PROGRESS] 리뷰 수집 중... 60/100
    [PROGRESS] 리뷰 수집 중... 80/100
    [PROGRESS] 리뷰 수집 중... 100/100
    [SU

In [6]:
import pandas as pd
df = pd.read_excel('./oliveyoung_data/oliveyoung_data_20251220_011339.xlsx', engine='openpyxl')

In [ ]:
df.head()

,country,major_category,sub_category,rank,product_name,brand,discount_rate,price,rating,ingredients,description,review_content,review_date,review_rating,review_detail_ratings,product_url
0,USA,Skincare,All Skincare,1,★2025 Awards★ Anua Niacinamide 10 TXA 4 Dark S...,Anua,59%,US$74.00,4.8,Contains 10% niacinamide to supply vitamin B3 ...,Provides clean and clear skin and moisture glo...,"The composition is very good, with three activ...",2025/12/16,4,"{""Absorbs quickly"": 3.0, ""Moisturizing"": 5.0, ...",https://global.oliveyoung.com/product/detail?p...
1,USA,Skincare,All Skincare,1,★2025 Awards★ Anua Niacinamide 10 TXA 4 Dark S...,Anua,59%,US$74.00,4.8,Contains 10% niacinamide to supply vitamin B3 ...,Provides clean and clear skin and moisture glo...,This product is so popular right now in the In...,2025/12/19,5,"{""Absorbs quickly"": 4.0, ""Moisturizing"": 4.0, ...",https://global.oliveyoung.com/product/detail?p...
2,USA,Skincare,All Skincare,1,★2025 Awards★ Anua Niacinamide 10 TXA 4 Dark S...,Anua,59%,US$74.00,4.8,Contains 10% niacinamide to supply vitamin B3 ...,Provides clean and clear skin and moisture glo...,Serums containing almost the same ingredients ...,2025/12/15,5,"{""Absorbs quickly"": 5.0, ""Moisturizing"": 4.0, ...",https://global.oliveyoung.com/product/detail?p...
3,USA,Skincare,All Skincare,1,★2025 Awards★ Anua Niacinamide 10 TXA 4 Dark S...,Anua,59%,US$74.00,4.8,Contains 10% niacinamide to supply vitamin B3 ...,Provides clean and clear skin and moisture glo...,The ingredients is good! It is sticky and feel...,2025/09/29,4,"{""Absorbs quickly"": 3.0, ""Moisturizing"": 5.0, ...",https://global.oliveyoung.com/product/detail?p...
4,USA,Skincare,All Skincare,1,★2025 Awards★ Anua Niacinamide 10 TXA 4 Dark S...,Anua,59%,US$74.00,4.8,Contains 10% niacinamide to supply vitamin B3 ...,Provides clean and clear skin and moisture glo...,I bought it when it was on sale and used it fo...,2025/08/31,5,"{""Absorbs quickly"": 3.0, ""Moisturizing"": 5.0, ...",https://global.oliveyoung.com/product/detail?p...


In [9]:
# 브랜드 종류 확인
df['brand'].value_counts()

brand
Anua            327
ROUND LAB       283
Torriden        200
WELLAGE         200
BIOHEAL BOH     130
REJURAN         118
Dr. Althea      110
AESTURA         100
S.NATURE        100
d'Alba          100
Centellian24    100
SKIN1004        100
medicube        100
celimax         100
beplain         100
ilso            100
VT              100
Dr.G            100
goodal           40
Name: count, dtype: int64

In [10]:
df['sub_category'].value_counts()

sub_category
All Skincare    2508
Name: count, dtype: int64